In [122]:
from math import exp
import pandas as pd
import numpy as np
from scipy.special import gamma as GAMMA

In [123]:
zeta, tau, gamma = 0.13583964, -0.31637652, 10.

In [124]:
def get_fractional_series(X0):
    """
    :param X0: 待计算分数阶的序列
    :return: 分数阶序列
    """
    len_x0 = len(X0)

    # 初始化数组
    xishu = np.zeros((len_x0, len_x0))
    Xr = np.zeros(len_x0)

    for k in range(len_x0):
        tmp = 0
        for i in range(k + 1):
            xishu[k, i] = GAMMA(zeta + k - i) / (GAMMA(k - i + 1) * GAMMA(zeta))
            tmp += xishu[k, i] * X0[i]
        Xr[k] = tmp

    return Xr

In [125]:
data = pd.read_excel("京津冀3省2005-2023使用版数据-2位小数.xlsx")
data = data.iloc[11:, :] # i=5
n = data.shape[0]
train_n = n-2
data

,电子及通信设备制造业,主营业务收入（亿元）,R&D活动人员折合全时当量（人年）,R&D经费内部支出（亿元）
11,2016,1.507327,3.299884,6.701289
12,2017,1.596237,3.125379,7.232092
13,2018,1.685147,2.950875,7.762178
14,2019,1.753019,2.940913,7.183381
15,2020,1.933047,3.291241,8.746418
16,2021,2.519186,3.359376,9.791547
17,2022,2.522894,4.411321,13.322350
18,2023,2.564832,4.867823,16.109319


In [126]:
# 第一列是年份，故先舍弃
X0 = data.iloc[:train_n, 1:].to_numpy()

X1 = np.array([get_fractional_series(X0[:,0]), get_fractional_series(X0[:,1]), get_fractional_series(X0[:,2])]).T
y1 = X1[:, 0] ** (1 - gamma)

Zy = (y1[1:] + y1[:-1]) / 2
M2 = (X1[1:, 1] + X1[:-1, 1]) / 2
M3 = (X1[1:, 2] + X1[:-1, 2]) / 2

In [127]:
# 构建 B
TAU = (np.exp(tau) - 1) / tau * np.exp(tau * (np.arange(1, train_n)))
ONE = np.ones(train_n - 1)
B = (1 - gamma) * np.array([-Zy, M2, M3, TAU, ONE]).T

# 构建 Y
Y = np.diff(y1)

# 最小二乘法
a, b2, b3, h1, h2 = np.dot(np.dot(np.linalg.inv(np.dot(B.T, B)), B.T), Y)
a, b2, b3, h1, h2

(-0.1706027395343204,
 0.00014340562340123242,
 -3.916114339357797e-05,
 -0.0008567870826304022,
 -9.646345406740247e-06)

In [128]:
# 求 mu
mu1 = (1-gamma)/(1+0.5*a*(1-gamma))
mu2 = (1-0.5*a*(1-gamma))/(1+0.5*a*(1-gamma))
mu3 = (h1*(1-gamma)*(exp(tau)-1))/(1+0.5*a*(1-gamma))/tau
mu4 = (h2*(1-gamma))/(1+0.5*a*(1-gamma))


In [129]:
# 这里重新获取下原数据，以实现所有数据的预测
# 第一列是年份，故先舍弃
X0 = data.iloc[:, 1:].to_numpy()

X1 = np.array([get_fractional_series(X0[:,0]), get_fractional_series(X0[:,1]), get_fractional_series(X0[:,2])]).T
# 求 hat y1(1为上标的那个haty^1)
hat_y1 = [y1[0]]
for m in range(2, n + 1):
    hat_ym = 0
    hat_ym += mu2 ** (m - 1) * y1[0]
    for v in range(1, m):
        hat_ym += mu2 ** (v - 1) * mu1 * (b2 * X1[m - v, 1] + b3 * X1[m - v, 2])
    for w in range(0, m - 1):
        hat_ym += mu2 ** w * (mu4 + mu3 * exp(tau * (m - w - 2)))
    hat_y1.append(hat_ym)
print(hat_y1)

[0.02489610490048321, 0.0060743519867794745, 0.0027688460599109292, 0.0014779284159720607, 0.0007953738735669889, 0.0004556058519966159, 6.632667606580893e-05, 1.7712945263338827e-05]


In [130]:
hat_x1 = np.array(hat_y1) ** (1 / (1 - gamma))
print(hat_x1)
hat_x0 = []
for k in range(2, n + 1):
    hat_x0_m = 0
    for j in range(0, k):
        hat_x0_m += (-1) ** j * GAMMA(zeta + 1) / (GAMMA(j + 1) * GAMMA(zeta - j + 1)) * hat_x1[k - j - 1]
    hat_x0.append(hat_x0_m)
print(hat_x0)

[1.5073275  1.7631034  1.92392891 2.0629227  2.20994245 2.35108243
 2.91243917 3.37262217]
[1.5583485752394481, 1.5959590791656382, 1.6431195785689812, 1.7131266379684005, 1.783170668162774, 2.277631771133037, 2.616192779003679]


In [131]:
original_x1 = data.iloc[1:, 1]
APE = np.abs(hat_x0 - original_x1) / original_x1
MAPE=np.mean(APE)

print(APE)
print(MAPE)

12    0.023736
13    0.052926
14    0.062691
15    0.113769
16    0.292164
17    0.097215
18    0.020025
Name: 主营业务收入（亿元）, dtype: float64
0.09464660267675852


In [132]:
pd.DataFrame(hat_x0).to_excel('FNGBM_hatx0_case2-11.xlsx')
pd.DataFrame(APE).to_excel('FNGBM_APE_case2-11.xlsx')